# Task
We want to achieve the following workflow:

Step 1. 
1. Write "Hello" in a file 

Step 2:
1. Check filename

**However, this is too simple for dflow. We want to write "Hello" in 10 different files.**

In [14]:
from typing import List

from dflow import Step, Workflow, argo_range
from dflow.python import OP, OPIO, Artifact, OPIOSign, PythonOPTemplate, Slices

** Hello OP **
- input: 
    - 'filename': str that is the file you want to get

In [15]:
class Hello(OP):
    def __init__(self):
        pass

    @classmethod
    def get_input_sign(cls):
        return OPIOSign({
            'filename': str
        })

    @classmethod
    def get_output_sign(cls):
        return OPIOSign({
            'out_art': Artifact(str)
        })

    @OP.exec_sign_check
    def execute(
            self,
            op_in: OPIO,
    ) -> OPIO:
        file_num = int(op_in["filename"].split('.')[0][1:])
        open(op_in["filename"], "w").write("Hello" * file_num)
        op_out = OPIO({
            'out_art': op_in["filename"]
        })
        return op_out

**Check OP**
It is to check the files that have been written and print the files' directory. `print(op_in["filename"])` 

In [16]:
class Check(OP):
    def __init__(self):
        pass

    @classmethod
    def get_input_sign(cls):
        return OPIOSign({
            'filename': Artifact(List[str])
        })

    @classmethod
    def get_output_sign(cls):
        return OPIOSign()

    @OP.exec_sign_check
    def execute(
            self,
            op_in: OPIO,
    ) -> OPIO:
        print(op_in["filename"])
        return OPIO()

And then we can use OP above to wrtite files repeatedly using Slices. This step is using Hello OP and python: 3.8 image.

**Slices**
We need to define two parameters in order to use this function, input_parameter and output_artifact, as well as argo_range.

In [17]:
wf = Workflow("slices")
hello = Step("hello",
            PythonOPTemplate(Hello, image="python:3.8",
                            slices=Slices("{{item}}",
                                        input_parameter=["filename"],
                                        output_artifact=["out_art"]
                                        )
                            ),
            parameters={"filename": [f"f{x}.txt" for x in range(10)]},
            with_param=argo_range(10))
check = Step("check",
            PythonOPTemplate(Check, image="python:3.8"),
            artifacts={"filename": hello.outputs.artifacts["out_art"]},
            )
wf.add(hello)
wf.add(check)
wf.submit();

Workflow has been submitted (ID: slices-z5s9v)


***by the way***

When we download artifacts online, we will see the following message:
`failed to get directory: stat /tmp/artifact3877355615/.dflow/34915d90-9d15-4956-a41e-596473177a64: not a directory`.
This problem is out of the usage of Slices, which is not supported well by Argo Workflows. Even though we cannot solve this problem, we can either download the artifact using dflow.download_artifact, or from Minio UI (http://localhost:9001 for localhost).